> edit

# import

In [ ]:
import itstgcntry
import torch

In [42]:
from torch_geometric_temporal.dataset import EnglandCovidDatasetLoader

# 예제7: iterative thresholded STGCN (IT-STGCN) with random missing

`-` data

In [66]:
loader = EnglandCovidDatasetLoader()

In [67]:
dataset = loader.get_dataset(lags=1)

In [69]:
train_dataset, test_dataset = itstgcntry.temporal_signal_split(dataset, train_ratio=0.8)

In [70]:
mindex = itstgcntry.rand_mindex(train_dataset,mrate=0.5)

In [71]:
train_dataset_miss = itstgcntry.miss(EnglandCovidDatasetLoader,train_dataset,mindex,mtype='rand')

In [76]:
torch.tensor(dataset.edge_indices[0]).shape

torch.Size([2, 2158])

In [83]:
_a = [train_dataset_miss.edge_indices[j].tolist() for j in range(T-1)]

In [84]:
torch.tensor(_a[0]).shape

torch.Size([2, 2158])

In [85]:
 data_dict = {
            'edges':[train_dataset_miss.edge_indices[j].tolist() for j in range(T-1)],
            'node_ids':{'node'+str(i):i for i in range(N)}, 
            'FX':FX}

In [86]:
EnglandCovidDatasetLoader(data_dict).get_dataset(lags=lags)

TypeError: __init__() takes 1 positional argument but 2 were given

In [24]:
train_dataset_padded = itstgcntry.padding(EnglandCovidDatasetLoader,train_dataset_miss) # padding(train_dataset_miss,method='linear'와 같음)a

UnboundLocalError: local variable 'train_dataset_padded' referenced before assignment

`-` 학습

In [ ]:
learn(self,DatasetLoader=DatasetLoader,typ=1,filters=32,epoch=50,lr=0.01,num_timesteps_in = None, num_timesteps_out = None,frames = None):

In [ ]:
lrnr = itstgcn.ITStgcnLearner(train_dataset_padded)

In [ ]:
lrnr.learn(filters=4,epoch=5)

`-` 적합값

In [ ]:
#lrnr(train_dataset_padded) 
#lrnr(test_dataset)['yhat'].shape

- 실행하면 X,y,yhat 출력 

`-` 모형 평가 및 시각화

In [ ]:
evtor = itstgcn.Evaluator(lrnr,train_dataset_padded,test_dataset)

In [ ]:
fig = evtor.plot('--.',h=5,max_node=3,label='complete data',alpha=0.5) # max_nodes 는 1보다 커야함
fig.set_figwidth(12)
fig.tight_layout()
fig

# 예제8: iterative thresholded STGCN (IT-STGCN) with block missing

`-` data

In [ ]:
_data = itstgcn.load_data('./data/fivenodes.pkl')
_edges = torch.tensor(_data['edges']).nonzero().tolist()
_FX = _data['f'].tolist()
_node_ids = {'node1':0, 'node2':1, 'node3':2, 'node4':3, 'node5':4} 
data_dict = {'edges':_edges, 'node_ids':_node_ids, 'FX':_FX}

In [ ]:
loader = itstgcn.DatasetLoader(data_dict)
dataset = loader.get_dataset(lags=2)
train_dataset, test_dataset = itstgcn.temporal_signal_split(dataset, train_ratio=0.8)

In [ ]:
mindex = [list(range(10,100)),[],list(range(50,80)),[],[]]
train_dataset_miss = itstgcn.miss(train_dataset,mindex,mtype='block')
train_dataset_padded = itstgcn.padding(train_dataset_miss) # padding(train_dataset_miss,method='linear'와 같음)

`-` 학습

In [ ]:
lrnr = itstgcn.ITStgcnLearner(train_dataset_padded)

In [ ]:
lrnr.learn(filters=4,epoch=5)

`-` 적합값

In [ ]:
#lrnr(train_dataset_padded) 
#lrnr(test_dataset)['yhat'].shape

- 실행하면 X,y,yhat 출력 

`-` 모형 평가 및 시각화

In [ ]:
evtor = itstgcn.Evaluator(lrnr,train_dataset_padded,test_dataset)

In [ ]:
fig = evtor.plot('--.',h=5,max_node=3,label='complete data',alpha=0.5) # max_nodes 는 1보다 커야함
fig.set_figwidth(12)
fig.tight_layout()
fig